In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import random
import time

In [ ]:
X = np.load('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir/bert_embedding.npy')  # shape (13169, 50, 768)

In [ ]:
# Melihat sampel pertama
print("\nIsi sampel pertama (X[0]):")
print(X[0])

# Melihat bentuk dari sampel pertama
print("\nShape dari X[0]:", X[0].shape)


Isi sampel pertama (X[0]):
[[-0.07174402 -0.14919606 -0.3100342  ...  0.44744673  0.22115424
  -0.21834277]
 [-0.81744933 -0.06987834 -0.7015157  ... -0.44357267  2.1919928
  -0.37429425]
 [-1.2884893   0.17520303 -0.82770914 ... -0.05995975  1.9407874
  -0.9524555 ]
 ...
 [-0.8777712  -0.7126367  -0.00831613 ...  0.48529652 -0.09414981
   0.7734433 ]
 [-0.30895013 -0.59683347  0.02157543 ...  0.43728426  0.46779865
   0.8894611 ]
 [-0.37264577 -0.65284747 -0.0693199  ...  0.22805968  1.0029004
   0.9099922 ]]

Shape dari X[0]: (50, 768)


In [ ]:
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir/preprocessed_data.csv')

In [ ]:
y = y.drop(columns=['Tweet']).values

In [ ]:
# Melihat sampel pertama
print("\nIsi sampel pertama (y[0]):")
print(y[0])

# Melihat bentuk dari sampel pertama
print("\nShape dari y[0]:", y[0].shape)


Isi sampel pertama (y[0]):
[1 1 1 0 0 0 0 0 1]

Shape dari y[0]: (9,)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
search_space = {
    'epochs': [20, 30, 40],
    'units': [10, 20, 30, 40, 50],
    'learning_rate': [5e-1, 1e-1, 1e-2],
    'batch_size': [128, 192, 256]
}

In [ ]:
def build_model(units, learning_rate):
    model = Sequential()
    model.add(Input(shape=(50, 768)))
    model.add(Bidirectional(GRU(units, return_sequences=False)))
    model.add(Dense(9, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    return model

In [ ]:
n_iterations = 40
results = []

print("Mulai Random Search Tuning...\n")

for i in range(n_iterations):
    # Pilih kombinasi hyperparameter secara acak
    params = {
        'epochs': random.choice(search_space['epochs']),
        'units': random.choice(search_space['units']),
        'learning_rate': random.choice(search_space['learning_rate']),
        'batch_size': random.choice(search_space['batch_size'])
    }

    print(f"Iterasi {i+1}/{n_iterations} ({int((i+1)/n_iterations*100)}%) - Hyperparams: {params}")

    # Bangun dan latih model
    model = build_model(params['units'], params['learning_rate'])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        verbose=1  # suppress training output
    )

    # Evaluasi di data validasi
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=1)

    # Simpan hasil
    results.append({
        'iteration': i+1,
        'params': params,
        'val_loss': val_loss,
        'val_accuracy': val_acc
    })

    print(f"  => Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}\n")

Mulai Random Search Tuning...

Iterasi 1/40 (2%) - Hyperparams: {'epochs': 20, 'units': 50, 'learning_rate': 0.01, 'batch_size': 192}
Epoch 1/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - binary_accuracy: 0.8036 - loss: 0.4085 - val_binary_accuracy: 0.8754 - val_loss: 0.2842
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - binary_accuracy: 0.8833 - loss: 0.2688 - val_binary_accuracy: 0.8899 - val_loss: 0.2599
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - binary_accuracy: 0.9037 - loss: 0.2307 - val_binary_accuracy: 0.8906 - val_loss: 0.2524
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - binary_accuracy: 0.9124 - loss: 0.2109 - val_binary_accuracy: 0.8952 - val_loss: 0.2480
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - binary_accuracy: 0.9228 - loss: 0.1899 - val_binary_accuracy: 0.8968 - val_loss: 0.2494
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - binary_accuracy: 0.9344 - loss: 0.1660 - val_binary_accuracy: 0.8969 - val_loss: 0.2485
Epoch 7/20
55/55 ━━━

In [ ]:
results_sorted = sorted(results, key=lambda x: x['val_accuracy'], reverse=True)
print("5 Kombinasi Hyperparameter Terbaik:")
for r in results_sorted[:5]:
    print(f"Iterasi {r['iteration']} - Val Acc: {r['val_accuracy']:.4f} - Params: {r['params']}")

5 Kombinasi Hyperparameter Terbaik:
Iterasi 1 - Val Acc: 0.8935 - Params: {'epochs': 20, 'units': 50, 'learning_rate': 0.01, 'batch_size': 192}
Iterasi 25 - Val Acc: 0.8915 - Params: {'epochs': 40, 'units': 50, 'learning_rate': 0.01, 'batch_size': 128}
Iterasi 33 - Val Acc: 0.8899 - Params: {'epochs': 20, 'units': 30, 'learning_rate': 0.01, 'batch_size': 256}
Iterasi 5 - Val Acc: 0.8888 - Params: {'epochs': 40, 'units': 50, 'learning_rate': 0.01, 'batch_size': 192}
Iterasi 4 - Val Acc: 0.8887 - Params: {'epochs': 20, 'units': 10, 'learning_rate': 0.01, 'batch_size': 192}


In [ ]:
# Menyimpan model terbaik
best_model_index = np.argmax([r['val_accuracy'] for r in results])
best_params = results[best_model_index]['params']
best_model = build_model(best_params['units'], best_params['learning_rate'])
best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=0)
best_model.save('best_bi_gru_model.keras')